In [29]:
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical,pad_sequences
from tflearn.datasets import imdb
tf.reset_default_graph()

In [30]:
# load Dataset
train,test,_=imdb.load_data(path='../data/imdb/imdb.pkl',
                            n_words=10000,valid_portion=0.1)
trainX,trainY = train
testX,testY=test
# pad the sequence
trainX=pad_sequences(trainX,maxlen=100,value=0.)
testX=pad_sequences(testX,maxlen=100,value=0.)
# one-hot encoding
trainY=to_categorical(trainY,nb_classes=2)
testY=to_categorical(testY,nb_classes=2)

In [31]:
# 
print("size trainX", trainX.shape)
print("size testX",testX.shape)
print("size testY",testY.shape)
print("size trainY",trainY.shape)

size trainX (22500, 100)
size testX (2500, 100)
size testY (2500, 2)
size trainY (22500, 2)


In [32]:
trainX[0,:]

array([ 17,  25,  10, 406,  26,  14,  56,  61,  62, 323,   4,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0])

In [33]:
# build an embedding
network=input_data(shape=[None, 100], name='input')
network=tflearn.embedding(network,input_dim=10000,output_dim=128)


In [34]:
# build convnet
branch1=conv_1d(network,nb_filter=128,filter_size=3,padding='valid',
                activation='relu',regularizer='L2')
branch2=conv_1d(network,nb_filter=128,filter_size=4,padding='valid',
               activation='relu',regularizer='L2')
branch3=conv_1d(network,nb_filter=128,filter_size=5,padding='valid',
               activation='relu',regularizer='L2')
network=merge([branch1,branch2,branch3],mode='concat',axis=1)
network=tf.expand_dims(network,2)
network=global_max_pool(network)
network=dropout(network,0.5)
network=fully_connected(network,2,activation='softmax')


In [35]:
network=regression(network,optimizer='adam',learning_rate=0.001,
                  loss='categorical_crossentropy',name='target')

In [36]:
#train
model=tflearn.DNN(network,tensorboard_verbose=0)
model.fit(trainX,trainY,n_epoch=5,shuffle=True,validation_set=(testX,testY),
         show_metric=True,batch_size=32)

Training Step: 3519  | total loss: 0.10474 | time: 124.551s
| Adam | epoch: 005 | loss: 0.10474 - acc: 0.9687 -- iter: 22496/22500
Training Step: 3520  | total loss: 0.10091 | time: 126.885s
| Adam | epoch: 005 | loss: 0.10091 - acc: 0.9718 | val_loss: 0.55239 - val_acc: 0.8132 -- iter: 22500/22500
--


In [37]:
model.

AttributeError: 'DNN' object has no attribute 'summary'